# Работа с API GigaChat с применением разных методов и параметров генерации.

## Авторизация

Регестрируемся на сайте developers.sber.ru, переходим в GigaChat API, подключаем тариф Freemium, переходим на вкладку "Настройка API", копируем Client ID и ключ авторизации в вкладку "Секреты" левой панели Colab для хранения ключей

In [ ]:
from google.colab import userdata
client_id = userdata.get('SBER_ID')
auth = userdata.get('SBER_AUTH')

Получим токен доступа для авторизации запросов к *API*

In [ ]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    rq_uid = str(uuid.uuid4())

    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    payload = {
        'scope': scope
    }

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [ ]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.UFsFz_E51fo1gaYDkvpFYooc-e7sxyb2ztPHM6eOHj3R38RJz2cBiCmbY5GKHleMixsdY7jnUPLjRMWFbd1oXWc9IkcLZA1te-JzIBtdE6800vL__El6maWP5fs_fwolGEsOP8S8DfLHwHPDL1L-X6LSiB3hOAYDo1uJSh-y47XcQvJpUHw-J3YcWzBwM8JIQEeK_geiKyTGTDMwFTa7poSlhxbrZMdjyHUR1EoxmoKX7t-SxgjfiAAwHnaa_LXcmleonYcfilG3s1x8CRsZI6Hl_iWel4KnJ6XMN1SdlApF4Ij7sPwZu9GeBtcAXc9jUSABOHHPmUaGxnYgaJS1xQ.CprcoktBGBR0OD8Tii0DKg.26G89fBrMqDRniKiWbx8x68kvN1yUSliFrU77g7WAYTJOXfWzyGUaQq5WPjituQdhunDOkhBw_XLwu64sQ2-0QbossUoqW_-XNisnHxFKR6yrS07LO3pTCW0FuNLT65ysvRpaqIWJwqkI75EHvvOHciDvmOUhlKkR3yqIzHowNPVBhBLiKnoNdVzx8oBK5KhYQ395ZGJyDF2lfJLvMbeAPC4gtjBDS0xQBM7xQOaZVeNvf99vXgbd1G4eiJtZq0tpxtS6v2n1GCCM8pZIe0RI6wCiSl7P2PITXwfyZST3zQsgbvC5srUT6DfJ72LBAUrX7bKVQmL73QqHcLtkzh-JpxVzBzdPWyDkA263gw-GUOUjPfCxHsT_9jz4yZ5npuSsdMiPdcHynQpKhCP7pclGWaJBzYLshOdJhCCb9c1EkuhVCpPBHErPuZOMV-HH_EIvJKuqn4NkRe3v-88LbTP-OOHteD7Y8jRBlkDDNKqDiMaW9xvV_pJK9ctygjKpXHDsiAV4waZxQZXE-dTKGdUvxFnoYumF

Получим список моделей

In [ ]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"Embeddings","object":"model","owned_by":"salutedevices","type":"embedder"},{"id":"Embeddings-2","object":"model","owned_by":"salutedevices","type":"embedder"},{"id":"EmbeddingsGigaR","object":"model","owned_by":"salutedevices","type"

Функция для отправки запроса к API чата и получения ответа

In [ ]:
import requests
import json


def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 512,
        "repetition_penalty": 1,
        "update_interval": 0
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Произошла ошибка: {str(e)}")
        return -1

Составим различные промпты использую методы: zero-shot, few-shot, chain-of-thought, self-consistency.

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("Методы для создания промптов:\n")

print("1. Zero-shot:")
zero_shot_prompt = "Объясни, что такое искусственный интеллект простыми словами."
response = get_chat_completion(giga_token, zero_shot_prompt)
print(f"Запрос: {zero_shot_prompt}")
print(f"Ответ: {response.json()['choices'][0]['message']['content']}\n")

print("2. Few-shot:")
few_shot_prompt = """
Классифицируй настроение текста как позитивное, негативное или нейтральное.

Примеры:
Текст: "Этот фильм просто потрясающий!"
Настроение: позитивное

Текст: "Ужасное обслуживание, никогда больше не приду."
Настроение: негативное

Текст: "Сегодня обычный день."
Настроение: нейтральное

Текст: "Обожаю эту новую функцию в приложении!"
Настроение:
"""

response = get_chat_completion(giga_token, few_shot_prompt)
print(f"Запрос: {few_shot_prompt.split('Текст: ')[-1].split('Настроение:')[0]}")
print(f"Ответ: {response.json()['choices'][0]['message']['content']}\n")

print("3. Chain-of-Thought:")
cot_prompt = """
Задача: У Пети было 10 рублей. Он потратил 4 рубля на конфеты,
а затем нашел 2 рубля. Сколько денег у Пети теперь?

Давай решим по шагам на русском:
"""

response = get_chat_completion(giga_token, cot_prompt)
print(f"Запрос: {cot_prompt.split('Задача:')[1].split('?')[0]}?")
print(f"Ответ: {response.json()['choices'][0]['message']['content']}\n")

print("4. Self-consistency:")
self_consistency_prompt = """
Реши логическую задачу несколькими разными способами и выбери наиболее последовательный ответ.

Задача: В корзине лежат яблоки. Если раздать по 2 яблока 3 людям, останется 1 яблоко.
Если раздать по 3 яблока 2 людям, не хватит 1 яблока. Сколько яблок в корзине?

Реши на русском несколькими способами и выбери наиболее последовательный ответ :
"""

response = get_chat_completion(giga_token, self_consistency_prompt)
print(f"Запрос: {self_consistency_prompt.split('Задача:')[1].split('?')[0]}?")
print(f"Ответ: {response.json()['choices'][0]['message']['content']}\n")

Методы для создания промптов:

1. Zero-shot:
Запрос: Объясни, что такое искусственный интеллект простыми словами.
Ответ: Искусственный интеллект (ИИ) — это когда компьютеры и программы начинают «думать» и действовать как человек: понимают, учатся, решают задачи, общаются и даже принимают решения, как мы с вами. Например, голосовой помощник в телефоне, который помогает найти нужную информацию или подсказывает маршрут, — это и есть искусственный интеллект. Или умные машины, которые сами управляют автомобилями, распознают лица и помогают врачам ставить диагнозы. ИИ делает жизнь проще и удобнее.

2. Few-shot:
Запрос: "Обожаю эту новую функцию в приложении!"

Ответ: **Текст:** «Обожаю эту новую функцию в приложении!»

**Настроение:** позитивное

3. Chain-of-Thought:
Запрос:  У Пети было 10 рублей. Он потратил 4 рубля на конфеты,
а затем нашел 2 рубля. Сколько денег у Пети теперь?
Ответ: Решим задачу пошагово:

1. Изначально у Пети было $10$ рублей.
2. Петя потратил $4$ рубля на конфеты, зна

Тестирование разных параметров генерации

In [ ]:
def get_chat_completion_with_params(auth_token, user_message, temperature=1.0, top_p=0.1, max_tokens=512):
    """
    Модифицированная функция с настраиваемыми параметрами генерации
    """
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [{"role": "user", "content": user_message}],
        "temperature": temperature,
        "top_p": top_p,
        "n": 1,
        "stream": False,
        "max_tokens": max_tokens,
        "repetition_penalty": 1,
        "update_interval": 0
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Произошла ошибка: {str(e)}")
        return -1

test_prompt = "Напиши короткое описание кота (2-3 предложения)."

print("1. Тестирование разных значений temperature:")
temperatures = [0.1, 0.5, 1.0, 1.5]
for temp in temperatures:
    response = get_chat_completion_with_params(giga_token, test_prompt, temperature=temp)
    content = response.json()['choices'][0]['message']['content']
    print(f"Temperature {temp}: {content[:]}\n")

print("2. Тестирование разных значений Top_p:")
top_p_values = [0.1, 0.5, 0.9]
for top_p in top_p_values:
    response = get_chat_completion_with_params(giga_token, test_prompt, top_p=top_p)
    content = response.json()['choices'][0]['message']['content']
    print(f"Top_p {top_p}: {content[:]}\n")

print("3. Тестирование разных значений Max_tokens:")
max_tokens_values = [50, 100, 200]
for max_toks in max_tokens_values:
    response = get_chat_completion_with_params(giga_token, test_prompt, max_tokens=max_toks)
    content = response.json()['choices'][0]['message']['content']
    print(f"Max_tokens {max_toks}: {content}\n")

Тестирование параметров генерации:

1. Тестирование разных значений temperature:
Temperature 0.1: Это спокойный и ласковый кот с густой серой шерстью и выразительными зелёными глазами. Он любит проводить время, уютно устроившись на коленях хозяина, мурлыча в ответ на поглаживания и нежные слова. В свободное время предпочитает лениво наблюдать за происходящим из окна или спокойно отдыхать в своём любимом кресле.

Temperature 0.5: Это спокойный и ласковый кот с густой шерстью серо-белого окраса, который обожает проводить время на коленях у хозяина, мурлыча в ответ на поглаживания. Он уравновешен, не слишком активен, но всегда рад поиграть с любимой игрушкой или просто полежать рядом.

Temperature 1.0: Это спокойный и ласковый кот с густой серой шерстью и выразительными зелёными глазами, который обожает проводить время на подоконнике, наблюдая за птицами, и уютно устроиться рядом с хозяином на диване.

Temperature 1.5: Это спокойный и ласковый кот с густой серой шерстью и выразительными з

Адаптации промптов под конкретные задачи, использование ролей

In [ ]:
print("1. Техническая документация:")
tech_prompt = """
Ты - senior Python разработчик. Напиши документацию для функции,
которая сортирует список чисел методом пузырьковой сортировки.

Требования к документации:
- Краткое описание алгоритма
- Параметры функции
- Возвращаемое значение
- Пример использования

Функция:
def bubble_sort(numbers):
    n = len(numbers)
    for i in range(n):
        for j in range(0, n - i - 1):
            if numbers[j] > numbers[j + 1]:
                numbers[j], numbers[j + 1] = numbers[j + 1], numbers[j]
    return numbers
"""

response = get_chat_completion(giga_token, tech_prompt)

print(response.json()['choices'][0]['message']['content'] + "\n")

print("2. Креативное письмо:")
creative_prompt = """
Ты - писатель-фантаст. Напиши короткий отрывок (3-4 предложения)
о городе будущего, где технологии и природа сосуществуют в гармонии.
Используй яркие образы и метафоры.
"""

response = get_chat_completion(giga_token, creative_prompt)

print(response.json()['choices'][0]['message']['content'] + "\n")

print("3. Образовательный контент:")
education_prompt = """
Объясни понятие "фотосинтез" как будто ты учитель биологии для учеников 5 класса.
Используй простые аналогии и примеры из повседневной жизни.
Ограничься 5-6 предложениями.
"""

response = get_chat_completion(giga_token, education_prompt)

print(response.json()['choices'][0]['message']['content'] + "\n")

Промпты для разных сценарий

1. Техническая документация:
## Документация функции `bubble_sort`

### Описание алгоритма
**Пузырьковая сортировка (Bubble Sort)** — простой алгоритм сортировки, который многократно проходит по списку чисел и сравнивает соседние элементы. Если текущий элемент больше следующего, они меняются местами. После каждого прохода самый большой элемент "всплывает" на своё место в конце списка, поэтому алгоритм и получил название «пузырьковая сортировка».

### Параметры функции
- `numbers` (список чисел) — список целых или вещественных чисел, который необходимо отсортировать.

### Возвращаемое значение
- отсортированный список чисел в порядке возрастания.

### Пример использования
```python
# Пример использования функции bubble_sort
numbers = [64, 34, 25, 12, 22, 11, 90]
sorted_numbers = bubble_sort(numbers)

# Вывод отсортированного списка
print(sorted_numbers)
# [11, 12, 22, 25, 34, 64, 90]
```

### Примечание
- Алгоритм пузырьковой сортировки имеет сложность $O(n^

Сравнение различных системных промптов

In [ ]:
def get_chat_completion_with_system(auth_token, system_message, user_message):
    """
    Функция с поддержкой системных промптов
    """
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        "temperature": 0.7,
        "top_p": 0.9,
        "max_tokens": 500
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Произошла ошибка: {str(e)}")
        return -1

system_prompts = {
    "эксперт": "Ты - эксперт по кибербезопасности с 10-летним опытом. Отвечай профессионально и технически точно.",
    "помощник": "Ты - дружелюбный AI-помощник. Отвечай просто и понятно, используя аналогии.",
    "аналитик": "Ты - аналитик данных. Структурируй ответы логически, используй нумерацию и выделяй ключевые моменты."
}

user_question = "Что такое двухфакторная аутентификация и почему она важна?"

print("Сравнение системных промптов:")
for role, system_msg in system_prompts.items():
    response = get_chat_completion_with_system(giga_token, system_msg, user_question)
    content = response.json()['choices'][0]['message']['content']
    print(f"\n--- {role.upper()} ---")
    print(f"{content[:]}")



Системные промпты

Сравнение системных промптов:

--- ЭКСПЕРТ ---
Двухфакторная (или многофакторная) аутентификация (MFA, от англ. Multi-Factor Authentication) — это метод повышения безопасности учетных записей пользователя путем добавления второго или дополнительного уровня проверки личности помимо традиционного пароля.

В общем случае, процесс MFA состоит из трёх основных факторов:
- **Знание** – пароль, PIN-код или секретный вопрос/ответ;
- **Владение** – физический объект (например, смартфон, аппаратный токен), на котором установлено приложение-аутентификатор, SIM-карта устройства связи или USB-токен;
- **Присутствие** – биометрические данные (отпечатки пальцев, лицо, радужка глаза, голосовые шаблоны).

При использовании двухфакторной аутентификации для входа в систему пользователь должен предоставить не только свой традиционный пароль (что является первым фактором), но также подтвердить свою личность через второй фактор: например, ввести одноразовый код (OTP, One Time Password), о